In [1]:
import os
import json
import cv2
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from tensorflow.keras.preprocessing import image
from tensorflow import keras
from tensorflow.keras.applications import resnet50, ResNet50
from tensorflow.keras.preprocessing import image


모델 load, 이미지 데이터 폴더 경로설정

In [2]:
json_path = 'E:/AInotes/자세교정/모델학습/label_data.json'
direction = 'face'
exercise_name = 'LyingLegRaise'
test_folder = f'E:/AI/dataset_skeleton_sep/{direction}/{exercise_name}/test'
exercise_name = 'LyingLegRaise'.lower()
model = tf.keras.models.load_model(f'E:/AImodel/models/Multilabel/VGG16/{exercise_name}.h5')


함수정의

In [3]:
def resize_img(image_paths):
    images_resized = []  # 리사이즈된 이미지를 저장할 리스트
    for image_path in image_paths:
        image = cv2.imread(image_path)  # 각 이미지 경로로부터 이미지를 읽음
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # BGR에서 RGB로 색상 변환
        image_resized = cv2.resize(image, (128, 128))  # 이미지 리사이즈
        images_resized.append(image_resized)  # 결과 리스트에 추가
    images_resized = np.array(images_resized) / 255.0  # numpy 배열로 변환 및 정규화
    return images_resized
    

def process_dataset(root_folder):
    image_paths = []
    label_data = []

    for roots, dirs, files in os.walk(root_folder):
        for file in files:
            if file.endswith('.jpg'):
                # 파일 이름 분석을 위해 숫자만 추출
                prefix = file[0:3]

                # 접두사에 따른 레이블 할당
                label = prefix_to_label.get(prefix)

                # 유효한 레이블이 있는 경우에만 리스트에 추가
                if label is not None:
                    image_paths.append(os.path.join(roots, file))
                    label_data.append(label)

    return image_paths, label_data


def apply_sliding_window(sequence, window_size=5, method='mean'):
    smoothed_sequence = []
    for i in range(len(sequence)):
        start_idx = max(0, i - window_size // 2)
        end_idx = min(len(sequence), i + window_size // 2 + 1)
        window = sequence[start_idx:end_idx]
        if method == 'mean':
            smoothed_value = np.mean(window)
        elif method == 'median':
            smoothed_value = np.median(window)
        else:
            raise ValueError("Method should be 'mean' or 'median'")
        smoothed_sequence.append(smoothed_value)
    return smoothed_sequence


def load_json_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)


def get_prefix_to_label(file_path, exercise_name):
    data = load_json_file(file_path)
    result = (data.get(exercise_name))
    prefix = data.get(exercise_name)['prefix']
    label = data.get(exercise_name)['last_path']
    prefix_to_label = dict(zip(prefix, label))
    return prefix_to_label
    

In [4]:
prefix_to_label = get_prefix_to_label(json_path, exercise_name)

test_image_paths, test_label_data = process_dataset(test_folder)
print(len(test_image_paths), len(test_label_data))

3406 3406


In [5]:
test_image_resized = resize_img(test_image_paths)
# 모델 예측
predictions = model.predict(test_image_resized)

# 임계값 설정 (예: 0.5)
threshold = 0.5
predictions_binary = (predictions > threshold).astype(int)

# 각 레이블에 대한 정확도 계산
accuracy_per_label = np.mean(predictions_binary == test_label_data, axis=0)

# 각 레이블별 정확도 출력
for i, accuracy in enumerate(accuracy_per_label):
    print(f"레이블 {i+1}의 정확도: {accuracy}")

# 전체 정확도도 여전히 중요할 수 있으므로, 이를 계산합니다.
overall_accuracy = np.mean(predictions_binary == test_label_data)
print(f"전체 정확도: {overall_accuracy}")


107/107 [==============================] - 8s 42ms/step
레이블 1의 정확도: 0.9128009395184967
레이블 2의 정확도: 0.91015854374633
레이블 3의 정확도: 0.88285378743394
레이블 4의 정확도: 0.9495008807985907
전체 정확도: 0.9138285378743394


In [6]:
# 예측 결과를 전치(Transpose)하여 각 레이블의 모든 예측 결과를 연속으로 배열
transposed_predictions = np.transpose(predictions)

smoothed_predictions = []
for label_predictions in transposed_predictions:
    # 각 레이블 별로 예측값에 슬라이딩 윈도우 적용
    smoothed = apply_sliding_window(label_predictions, window_size=5, method='mean')
    smoothed_predictions.append(smoothed)

# 슬라이딩 윈도우 적용 결과를 다시 전치하여 원래 형태로 복원
smoothed_predictions = np.transpose(smoothed_predictions)

# Apply threshold to smoothed predictions
smoothed_predictions_binary = (smoothed_predictions > threshold).astype(int)

# Calculate accuracy for each label in smoothed predictions
smoothed_accuracy_per_label = np.mean(smoothed_predictions_binary == test_label_data, axis=0)

# Print accuracy for each label
for i, accuracy in enumerate(smoothed_accuracy_per_label):
    print(f"레이블 {i+1}의 보정된 정확도: {accuracy}")

# Calculate overall accuracy for smoothed predictions
smoothed_overall_accuracy = np.mean(smoothed_predictions_binary == test_label_data)
print(f"보정된 전체 정확도: {smoothed_overall_accuracy}")


레이블 1의 보정된 정확도: 0.9797416324133882
레이블 2의 보정된 정확도: 0.9732824427480916
레이블 3의 보정된 정확도: 0.9600704638872578
레이블 4의 보정된 정확도: 0.9856136230182032
보정된 전체 정확도: 0.9746770405167352


In [7]:
# # 각 예측 결과에 대해 슬라이딩 윈도우 적용
# smoothed_predictions = []
# for prediction in predictions:
#     smoothed = apply_sliding_window(prediction.flatten(), window_size=5, method='mean')  # 예측 결과가 2차원일 수 있으므로 flatten() 호출
#     smoothed_predictions.append(smoothed)

# # Apply threshold to smoothed predictions
# smoothed_predictions_binary = (np.array(smoothed_predictions) > threshold).astype(int)

# # Calculate accuracy for each label in smoothed predictions
# smoothed_accuracy_per_label = np.mean(smoothed_predictions_binary == test_label_data, axis=0)

# # Print accuracy for each label
# for i, accuracy in enumerate(smoothed_accuracy_per_label):
#     print(f"레이블 {i+1}의 보정된 정확도: {accuracy}")

# # Calculate overall accuracy for smoothed predictions
# smoothed_overall_accuracy = np.mean(smoothed_predictions_binary == test_label_data)
# print(f"보정된 전체 정확도: {smoothed_overall_accuracy}")
